# `quanto` integration in `transformers`

Welcome to this tutorial where we showcase how to use `quanto` library and `transformers` to quantize any model in 8, 4, even 2 bit precision on GPU / CPU and MPS device! Let's get started 🔥

## Download requirements

First, install the dependencies below to get started. As these features are available on the `main` branches only, we need to install the libraries below from source.

In [ ]:
!pip install -U -q git+https://github.com/huggingface/transformers.git
!pip install -U -q quanto
!pip install -U -q accelerate
!pip install -q datasets
!pip install --upgrade transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Basic usage

### Quantize the model

You can quantize a model by passing a `QuantoConfig` object in the from_pretrained method !

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig
import torch

#model_id = "bigscience/bloom-560m"
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

quantization_config = QuantoConfig(weights="int4")

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32, quantization_config=quantization_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

If you print the model, you will see that most of the `nn.Linear` layers are replaced by `bnb.nn.Qlinear` layers! You can also see that the scale has the same dtype as the orignal model.

Once loaded, run a prediction as you would do it with a classic model

In [ ]:
text = "Hello my name is"
device = "cuda"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/quanto/library/ops.py:66: UserWarning: An exception was raised while calling the optimized kernel for quanto::unpack: Error building extension 'quanto_cuda': [1/5] /usr/local/cuda/bin/nvcc --generate-dependencies-with-compile --dependency-output gemm_cuda.cuda.o.d -DTORCH_EXTENSION_NAME=quanto_cuda -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.10/dist-packages/torch/include -isystem /usr/local/lib/python3.10/dist-packages/torch/include/torch/csrc/api/include -isystem /usr/local/lib/python3.10

Hello my name is Rachel and I am a 20 year old college student. I am a junior majoring in communications
